## Project outline
### Could it have been predicted?

In this project we will develop a series of safety pharmacology models using python based cheminformatic tools such as Rdkit, Scikitlearn, and pytorch. We want to see if machine learning cheminformatic toxicity models can single out the molecules that were ultimately removed from the market. I have collected a database of drugs that were withdrawn from the market due to toxicity for various reasons. This can be combined with a database of currently available medications to form a test set which can be used to test a model designed to predict toxicity. Many of the drugs that have been withdrawn over the years were withdrawn due to hepatotoxicity or DILI (drug induced liver injury) so I have collected a dataset of molecules with BSEP binding values to develop a model capable of predicting hepatoxicity. This will be the first model. If time allows, we will also gather data on other toxicities responsible for drug withdrawal such as binding the HERG (IKr) associated protein and potentially other secondary pharmacology assay targets such as Gprotein-coupledreceptors (GPCRs), enzymes, kinases, nuclear hormone receptors, ion channels and transporters.

### References
Assay Targets:

Jenkinson, S., et al., A practical guide to secondary pharmacology in drug discovery. Journal of Pharmacological and Toxicological Methods, 2020. 105.

BSEP Database:

AbdulHameed, M.D.M., R. Liu, and A. Wallqvist, Using a Graph Convolutional Neural Network Model to Identify Bile Salt Export Pump Inhibitors. ACS Omega, 2023. 8(24): p. 21853-21861.

Dataset of Withdrawn drugs:

Siramshetty, V.B., et al., WITHDRAWN--a resource for withdrawn and discontinued drugs. Nucleic Acids Res, 2016. 44(D1): p. D1080-6.

Onakpoya, I. J., Heneghan, C. J., & Aronson, J. K. (2016). Post-marketing withdrawal of 462 medicinal products because of adverse drug reactions: a systematic review of the world literature. BMC Medicine, 14, 10.

## Import Dependencies

### General
* Pandas
* Numpy
* Seaborn

### Datasets
* TDC Tox

### RDKit Modules
* AllChem
* rdMolDescriptors
* IPythonConsole
* Draw
* DataStructs
* Butina

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
#---------------------- Therapeutic Drug Commons (TDC data) from https://tdcommons.ai/single_pred_tasks/tox/#dili-drug-induced-liver-injury
from tdc.single_pred import Tox
#---------------------- RDKit packages
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
#-
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import ensemble
from sklearn.model_selection import ShuffleSplit, cross_validate,train_test_split

## Data cleaning

### Reading, converting to pandas

Read TDC Tox DILI Dataset & convert to Pandas dataframe.\
Rename columns to be more human-readable.

In [2]:
# This is Xu dataset
tox_data = Tox(name = 'DILI')
tox_df = tox_data.get_data()
tox_df.columns = ["X", "SMILES", "DILI?"]

Found local copy...
Loading...
Done!


In [10]:
Hameed_df = pd.read_excel("Transformed_Data/Hameed_BSEP.xlsx", sheet_name=1, usecols = range(1, 3))

print(Hameed_df.head(2))

                                              SMILES  BSEP ACTIVITY
0  CC[C@H]1C(=O)N(C)[C@H](C)C(=O)N(CC)[C@@H](C(C)...              1
1  CCC1C(=O)N(C)C(C)C(=O)N(CC)C(C(C)C)C(=O)NC(C(C...              1


In [11]:
onakpoya_df = pd.read_csv('Transformed_Data/Onakpoya_Drugs.csv', skiprows=[0]) # Read the table as csv
onakpoya_df = onakpoya_df.filter(['Medicinal product', 'Reason for withdrawal']) # Drop irrelevant columns
onakpoya_df = onakpoya_df[onakpoya_df['Reason for withdrawal'].str.endswith('Liver', na = False)] # Drop non-DILI related withdrawal
onakpoya_df = onakpoya_df.replace({'‡':''}, regex=True) # Remove uninterpretable characters
onakpoya_df['Medicinal product'] = onakpoya_df['Medicinal product'].str.partition(' ')[0] # Only keep first word of drug name

## Drop multiword rows
# multiword_rows = onakpoya_df[onakpoya_df['Medicinal product'].str.split().str.len() > 1].index
# onakpoya_df = onakpoya_df.drop(multiword_rows, axis = 0)

import pubchempy as pcp

onakpoya_df['pcp_result'] = onakpoya_df['Medicinal product'].map(lambda x: pcp.get_compounds(identifier=x, namespace='name')) # Get pubchem CID for each compound

#onakpoya_df['pcp_result'] = onakpoya_df['pcp_result'].str.partition(' ')[0] # Only keep first CID

print(onakpoya_df.head(2))


KeyboardInterrupt: 

In [ ]:
onakpoya_df.to_csv('Transformed_Data/attempto.csv')

In [ ]:
onakpoya_df2 = onakpoya_df

#onakpoya_df2.to_csv('Transformed_Data/attempt.csv')

# for compound in onakpoya_df2['pcp_result']:
#     print(onakpoya_df2['pcp_result'].isomeric_smiles)


### Append skeleton structures

Generate RDKit molecular structure and append skeleton diagram as a column to the dataset.

In [4]:
# Get RDKit molecular structure
from rdkit.Chem import PandasTools
PandasTools.AddMoleculeColumnToFrame(tox_df, 'SMILES', 'Structure')
print(tox_df.head(2))

# Display RDKit molecule skeleton - Use Ipython to fix broken pandas functionality (doesn't display skeleton in HTML table)
# from IPython.display import HTML

       X                 SMILES  DILI?  \
0  187.0   CC(=O)OCC[N+](C)(C)C    0.0   
1  247.0  C[N+](C)(C)CC(=O)[O-]    0.0   

                                           Structure  
0  <rdkit.Chem.rdchem.Mol object at 0x0000021F2DE...  
1  <rdkit.Chem.rdchem.Mol object at 0x0000021F2DE...  


# Get Fingerprints

Define function 'generate_fingerprints'
Initialise empty list of Morgan fingerprints
for molecules in a given dataframe, generate their morgan fingerprints and append them to the dataframe
Reutrn appended dataframe as numpy array to analyse using 'shape'

Run generate_fingerprints on each molecule in the dataframe

Use shape to confirm success - First number should equal dataframe length


In [13]:
from rdkit.Chem import MolFromSmiles

def smiles_to_morgan_fingerprint(smiles):
    mol = MolFromSmiles(smiles)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 2048)
    return fingerprint

tox_df["M3FP"] = [smiles_to_morgan_fingerprint(smiles) for smiles in tox_df["SMILES"]]

print(tox_df)

              X                                             SMILES  DILI?  \
0         187.0                               CC(=O)OCC[N+](C)(C)C    0.0   
1         247.0                              C[N+](C)(C)CC(=O)[O-]    0.0   
2         298.0       O=C(NC(CO)C(O)c1ccc([N+](=O)[O-])cc1)C(Cl)Cl    0.0   
3         338.0                                    O=C(O)c1ccccc1O    0.0   
4         444.0                     CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1    0.0   
..          ...                                                ...    ...   
470  23663956.0           CCCC(CCC)C(=O)O.CCCC(CCC)C(=O)[O-].[Na+]    1.0   
471  23673837.0  CCCCC(CC)COC(=O)CC(C(=O)OCC(CC)CCCC)S(=O)(=O)[...    0.0   
472  54675785.0  C=C1c2cccc(O)c2C(O)=C2C(=O)C3(O)C(O)=C(C(N)=O)...    1.0   
473  54678501.0                             O=C1OC(C(O)CO)C(O)=C1O    0.0   
474  54680690.0  CN(C)C1C(=O)C(C(N)=O)=C(O)C2(O)C(=O)C3=C(O)c4c...    1.0   

                                             Structure  \
0    <rdkit.Chem.

In [58]:
morgan_df = tox_df["m3fp"].apply(pd.Series)

morgan_df.insert(len(morgan_df.columns), "DILI?", tox_df["DILI?"].astype(int)) # Insert "DILI?" column as the last column

morgan_df.columns = morgan_df.columns.astype(str) # Set all column titles to string - Required for model

print(morgan_df.head(2))

   0  1  2  3  4  5  6  7  8  9  ...  2039  2040  2041  2042  2043  2044  \
0  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   
1  0  0  0  0  0  0  0  0  0  0  ...     0     0     0     0     0     0   

   2045  2046  2047  DILI?  
0     0     0     0      0  
1     0     0     0      0  

[2 rows x 2049 columns]


In [59]:
# Instantiate Random Forest Regressor
model_rf = ensemble.RandomForestRegressor(random_state=42)

# Instantiate a dict of paramaters of RFR
params_rf = {
            'bootstrap': [True, False],
            'max_depth': [range(1, 4096, 10), None],
            'max_features': ['auto', 'sqrt', 'log2', 1],
            'min_samples_leaf': range(1, 8, 1),
            'min_samples_split': range(1, 8, 1),
            'n_estimators': range(10, 800, 10)
            }

X = morgan_df.iloc[:, 0:len(morgan_df.columns)-1] # Features
y = morgan_df["DILI?"].values # Labels

#model_rf.fit(X, y)

In [60]:
from sklearn.model_selection import RandomizedSearchCV

# Set up scoring methods for hyperparameter tuning
scoring = ['r2', 'neg_root_mean_squared_error']

# Enable multithreading functionality
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [61]:
# Run RandomizedSearchCV to optimise random forest hyperparameters
rf_cv = RandomizedSearchCV(model_rf, params_rf, cv = 5, n_iter=12, n_jobs=n_jobs, random_state=42, scoring='r2') #5-fold precedended in AbdulHameed

rf_cv.fit(X_test, y_test)

# Print scores
print("Tuned Logistic Regression Parameters: {}".format(rf_cv.best_params_)) 
print("Best score is {}".format(rf_cv.best_score_))

Tuned Logistic Regression Parameters: {'n_estimators': 710, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False}
Best score is 0.028006990579347368


In [62]:
from tpot import TPOTRegressor

pipeline_optimiser = TPOTRegressor(random_state = 42, n_jobs=n_jobs)

pipeline_optimiser.fit(X_train, y_train)
print(pipeline_optimiser.score(X_test, y_test))

RuntimeError: A pipeline has not yet been optimized. Please call fit() first.

In [ ]:
# Instantiate an XGBoost

model_gb = ensemble.GradientBoostingRegressor(random_state=42)

params_gb = {
            'n_estimators': range(0, 150, 10),
            'max_depth': [2, 3, 5, 10, 15],
            'learning_rate': [0.05, 0.1, 0.15, 0.20]
            }

from sklearn.model_selection import RandomizedSearchCV

gb_cv = RandomizedSearchCV(model_gb, params_gb, cv = 5, n_iter=32, n_jobs=n_jobs, random_state=42, scoring='r2')

gb_cv.fit(x, y)

# Print scores
print("Tuned Logistic Regression Parameters: {}".format(gb_cv.best_params_)) 
print("Best score is {}".format(gb_cv.best_score_))

Tuned Logistic Regression Parameters: {'n_estimators': 120, 'max_depth': 5, 'learning_rate': 0.05}
Best score is 0.3560953880782183


## Thanks To

https://www.youtube.com/watch?v=-oHqQBUyrQ0

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

https://github.com/PatWalters/practical_cheminformatics_posts/blob/main/solubility/literature_solubility_model.ipynb

https://leftwinglow.github.io/BachelorsProject/

https://github.com/gashawmg/Molecular-fingerprints/blob/main/Calculating%20molecular%20fingerprints%20available%20in%20RDkit%20.ipynb

https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/Avalon%20fingerprints%20for%20predictive%20modeling.ipynb